In [1]:
using Pkg; Pkg.activate(".")

  Activating environment at `~/.julia/dev/SymArrays/examples/Project.toml`


In [2]:
using Printf
using BenchmarkTools
using SymArrays
using SymArrays: SymIndexIter
println("Julia version $VERSION")

┌ Info: Precompiling SymArrays [c2b21f84-4a8e-4c96-99a2-2b79df568f75]
└ @ Base loading.jl:1342


Julia version 1.6.4


In [3]:
sumfirstinds(iter) = (s=0; for I in iter s+=first(Tuple(I)) end; s)
sumvals(iter) = reduce(+, iter);

In [4]:
ttt(iter) = (s=0; for ii in 1:length(iter) s+=first(SymArrays.ind2sub_symgrp(iter,ii)) end; s);

In [5]:
@btime ttt(SI)          setup=(SI=SymIndexIter(4,50))

  13.414 ms (0 allocations: 0 bytes)


3162510

In [6]:
@btime sumfirstinds(SI) setup=(SI=SymIndexIter(4,50))

  47.406 μs (0 allocations: 0 bytes)


3162510

In [7]:
for n in (10, 50, 200)
    n != 10 && println()
    println("n = $n:")
    for N in 1:4, M in (N,-N)
        iter = SymIndexIter(M,n)
        # check that number of elements is correct
        @assert count(x->true,iter) == length(iter)

        @printf("  M = %2d:", M)
        @btime sumfirstinds($iter)
    end
end

n = 10:
  M =  1:  2.173 ns (0 allocations: 0 bytes)
  M = -1:  2.827 ns (0 allocations: 0 bytes)
  M =  2:  68.361 ns (0 allocations: 0 bytes)
  M = -2:  62.818 ns (0 allocations: 0 bytes)
  M =  3:  136.056 ns (0 allocations: 0 bytes)
  M = -3:  94.582 ns (0 allocations: 0 bytes)
  M =  4:  579.967 ns (0 allocations: 0 bytes)
  M = -4:  243.629 ns (0 allocations: 0 bytes)

n = 50:
  M =  1:  1.820 ns (0 allocations: 0 bytes)
  M = -1:  1.821 ns (0 allocations: 0 bytes)
  M =  2:  1.563 μs (0 allocations: 0 bytes)
  M = -2:  1.793 μs (0 allocations: 0 bytes)
  M =  3:  2.630 μs (0 allocations: 0 bytes)
  M = -3:  2.453 μs (0 allocations: 0 bytes)
  M =  4:  47.694 μs (0 allocations: 0 bytes)
  M = -4:  39.839 μs (0 allocations: 0 bytes)

n = 200:
  M =  1:  2.173 ns (0 allocations: 0 bytes)
  M = -1:  2.172 ns (0 allocations: 0 bytes)
  M =  2:  23.604 μs (0 allocations: 0 bytes)
  M = -2:  29.116 μs (0 allocations: 0 bytes)
  M =  3:  37.905 μs (0 allocations: 0 bytes)
  M = -3:  37.

In [8]:
for S in (SymArray{(3,),Float64}(50,50,50),
          SymArray{(-3,),Float64}(50,50,50),
          SymArray{(3,-2,2),Float64}(2,2,2,3,3,4,4),
          SymArray{(3,2,2),Float64}(2,2,2,3,3,4,4),
          SymArray{(-3,-4),Float64}(8,8,8,5,5,5,5),
          SymArray{(3,-2,2),Float64}(2,2,2,30,30,4,4),
          SymArray{(3,2,2),Float64}(2,2,2,30,30,4,4),
          SymArray{(-3,-4),Float64}(20,20,20,5,5,5,5))
    @printf("typeof(S): %-54s, size(S): %s, length(S): %d, full length(S): %d\n", typeof(S), size(S), length(S), prod(size(S)))
    fill!(S,1.)
    @btime sumfirstinds($(eachindex(S)))
    @btime sumvals($S)
end

typeof(S): SymArray{(3,), Float64, 3, 1, Vector{Float64}}        , size(S): (50, 50, 50), length(S): 22100, full length(S): 125000
  2.631 μs (0 allocations: 0 bytes)
  150.491 μs (0 allocations: 0 bytes)
typeof(S): SymArray{(-3,), Float64, 3, 1, Vector{Float64}}       , size(S): (50, 50, 50), length(S): 19600, full length(S): 125000
  2.446 μs (0 allocations: 0 bytes)
  164.134 μs (0 allocations: 0 bytes)
typeof(S): SymArray{(3, -2, 2), Float64, 7, 3, Array{Float64, 3}}, size(S): (2, 2, 2, 3, 3, 4, 4), length(S): 120, full length(S): 1152
  395.612 ns (0 allocations: 0 bytes)
  1.846 μs (0 allocations: 0 bytes)
typeof(S): SymArray{(3, 2, 2), Float64, 7, 3, Array{Float64, 3}} , size(S): (2, 2, 2, 3, 3, 4, 4), length(S): 240, full length(S): 1152
  798.347 ns (0 allocations: 0 bytes)
  2.765 μs (0 allocations: 0 bytes)
typeof(S): SymArray{(-3, -4), Float64, 7, 2, Matrix{Float64}}    , size(S): (8, 8, 8, 5, 5, 5, 5), length(S): 280, full length(S): 320000
  294.761 ns (0 allocations: 0 b